In [3]:
import pandas as pd
from sodapy import Socrata
import pyspark
import json

conf = pyspark.SparkConf()
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)

client = Socrata("data.cityofnewyork.us", None)

23/05/03 21:44:08 WARN Utils: Your hostname, MacBook-Air-2.local resolves to a loopback address: 127.0.0.1; using 192.168.0.216 instead (on interface en0)
23/05/03 21:44:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/03 21:44:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/03 21:44:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/05/03 21:44:09 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [20]:
from pyspark.sql.functions import col

In [28]:
test_data_path = '/Users/maludee/repos/Big-Data-Project/data/Parking_Violations_Issued_-_Fiscal_Year_2023.csv'
precinct_data_path = "/Users/maludee/repos/Big-Data-Project/data/nypp.csv"

In [12]:
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(test_data_path)

df.repartition(1).write.mode('overwrite').parquet('tmp/parking_2023')

df = spark.read.parquet('/Users/maludee/repos/Big-Data-Project/notebooks/tmp/parking_2023/*.parquet')

In [13]:
df.limit(3).toPandas()

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
0,1484697303,JER1863,NY,PAS,06/10/2022,67,SDN,TOYOT,P,34330,...,BLK,0,2004,-,0,None,None,None,None,None
1,1484697315,KEV4487,NY,PAS,06/13/2022,51,SUBN,JEEP,K,34310,...,GRAY,0,2017,-,0,None,None,None,None,None
2,1484697625,H73NYD,NJ,PAS,06/19/2022,63,SDN,JEEP,N,30640,...,GRAY,0,0,-,0,None,None,None,None,None


In [17]:
df.printSchema()

root
 |-- Summons Number: long (nullable = true)
 |-- Plate ID: string (nullable = true)
 |-- Registration State: string (nullable = true)
 |-- Plate Type: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Violation Code: integer (nullable = true)
 |-- Vehicle Body Type: string (nullable = true)
 |-- Vehicle Make: string (nullable = true)
 |-- Issuing Agency: string (nullable = true)
 |-- Street Code1: integer (nullable = true)
 |-- Street Code2: integer (nullable = true)
 |-- Street Code3: integer (nullable = true)
 |-- Vehicle Expiration Date: integer (nullable = true)
 |-- Violation Location: integer (nullable = true)
 |-- Violation Precinct: integer (nullable = true)
 |-- Issuer Precinct: integer (nullable = true)
 |-- Issuer Code: integer (nullable = true)
 |-- Issuer Command: string (nullable = true)
 |-- Issuer Squad: string (nullable = true)
 |-- Violation Time: string (nullable = true)
 |-- Time First Observed: string (nullable = true)
 |-- Violation Coun

In [31]:
precinct_keys = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(precinct_data_path)
precinct_keys.show(50)

+--------------------+--------+-------------+---------------+
|            the_geom|Precinct|   Shape_Leng|     Shape_Area|
+--------------------+--------+-------------+---------------+
|MULTIPOLYGON (((-...|       1|80283.5387782|4.72864229826E7|
|MULTIPOLYGON (((-...|       5|18807.1249114|1.80945274385E7|
|MULTIPOLYGON (((-...|       6|24875.9642171|2.20179465474E7|
|MULTIPOLYGON (((-...|       7|17287.5444926| 1.8366669928E7|
|MULTIPOLYGON (((-...|       9|19772.5107407|2.13953862669E7|
|MULTIPOLYGON (((-...|      10|40227.2250545|2.72654964913E7|
|MULTIPOLYGON (((-...|      13|27719.2073747|2.95076562644E7|
|MULTIPOLYGON (((-...|      14|20973.8201315|2.05111281195E7|
|MULTIPOLYGON (((-...|      17|26719.6801963|2.22687682674E7|
|MULTIPOLYGON (((-...|      18|41625.4866799| 3.2250244174E7|
|MULTIPOLYGON (((-...|      19|30595.1089765|4.87915175453E7|
|MULTIPOLYGON (((-...|      20|27069.9614199|2.76738686519E7|
|MULTIPOLYGON (((-...|      22|32712.7637008|3.83122030723E7|
|MULTIPO

In [33]:
# use sql syntax to parse the nested json
df.createOrReplaceTempView("2023_violations")
query = """
select 
    `Violation Precinct` as violation_precinct,
    count(*) as count_violations
from 
    2023_violations
group by 1
order by 2 desc

"""
precincts = spark.sql(query)
precincts.toPandas().to_csv('violations_by_precinct.csv')
precincts.show()

+------------------+----------------+
|violation_precinct|count_violations|
+------------------+----------------+
|                 0|         5938384|
|                19|          331642|
|                13|          291119|
|                 6|          259154|
|               114|          254978|
|                14|          220291|
|                18|          201218|
|                 9|          186315|
|                 1|          174428|
|               109|          162445|
|               115|          148671|
|               108|          143484|
|                20|          139818|
|                70|          137570|
|                10|          126738|
|                84|          125162|
|                52|          121617|
|                66|          118049|
|               112|          117989|
|                17|          114972|
+------------------+----------------+
only showing top 20 rows



In [2]:
data_2014 = 'jt7v-77mi'
data_2015 = 'c284-tqph'
data_2016 = 'kiv2-tbus'
data_2016 = 'uu62-ifvu'
data_2017 = '2bnn-yakx'
data_2018 = 'a5td-mswe'
data_2019 = 'faiq-9dfq'
data_2020 = 'p7t3-5i9s'
data_2021 = 'kvfd-bves'
data_2022 = '7mxj-7a6y'
data_2023 = 'pvqr-7yc4'

main_data = "nc67-uf89"

In [3]:
# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
res_list = []
offset = 0
for i in range(2):    
    results = client.get(main_data, limit=50000, offset=offset)
    res_list.append(results)
    offset = offset + 50000

In [4]:
df = sc.parallelize(res_list).map(lambda x: json.dumps(x))
df = spark.read.json(df)

23/05/02 19:27:04 WARN TaskSetManager: Stage 0 contains a task of very large size (14993 KiB). The maximum recommended task size is 1000 KiB.


In [5]:
df.printSchema()

root
 |-- amount_due: string (nullable = true)
 |-- county: string (nullable = true)
 |-- fine_amount: string (nullable = true)
 |-- interest_amount: string (nullable = true)
 |-- issue_date: string (nullable = true)
 |-- issuing_agency: string (nullable = true)
 |-- judgment_entry_date: string (nullable = true)
 |-- license_type: string (nullable = true)
 |-- payment_amount: string (nullable = true)
 |-- penalty_amount: string (nullable = true)
 |-- plate: string (nullable = true)
 |-- precinct: string (nullable = true)
 |-- reduction_amount: string (nullable = true)
 |-- state: string (nullable = true)
 |-- summons_image: struct (nullable = true)
 |    |-- description: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- summons_number: string (nullable = true)
 |-- violation: string (nullable = true)
 |-- violation_status: string (nullable = true)
 |-- violation_time: string (nullable = true)



In [6]:
df.limit(3).toPandas()

23/05/02 19:27:12 WARN TaskSetManager: Stage 2 contains a task of very large size (14993 KiB). The maximum recommended task size is 1000 KiB.


,amount_due,county,fine_amount,interest_amount,issue_date,issuing_agency,judgment_entry_date,license_type,payment_amount,penalty_amount,plate,precinct,reduction_amount,state,summons_image,summons_number,violation,violation_status,violation_time
0,0,K,35,0,08/03/2018,TRAFFIC,None,PAS,35,0,FFC4982,060,0,NY,"(View Summons, http://nycserv.nyc.gov/NYCServW...",8661033100,FAIL TO DSPLY MUNI METER RECPT,None,08:16P
1,0,BX,35,0,04/27/2018,TRAFFIC,None,PAS,65,30,BTM5131,049,0,NY,"(View Summons, http://nycserv.nyc.gov/NYCServW...",8541152224,FAIL TO DSPLY MUNI METER RECPT,HEARING HELD-GUILTY,01:15P
2,0,BX,35,0,04/30/2018,TRAFFIC,None,PAS,95,60,HNY4992,047,0,NY,"(View Summons, http://nycserv.nyc.gov/NYCServW...",8541152327,FAIL TO DSPLY MUNI METER RECPT,None,01:14P


In [7]:
df.count()

23/05/02 19:27:15 WARN TaskSetManager: Stage 3 contains a task of very large size (14993 KiB). The maximum recommended task size is 1000 KiB.


100000

In [3]:
data_2023 = client.get('pvqr-7yc4', limit=50000, offset=0)



ConnectionError: HTTPSConnectionPool(host='data.cityofnewyork.us', port=443): Max retries exceeded with url: /resource/pvqr-7yc4.json?%24limit=50000&%24offset=0 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fdb3a8d2b80>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))